### Imports

In [29]:
import re
import string
import math
import itertools
from collections import defaultdict, Counter
from operator import itemgetter

### Alphabet

In [61]:
ALPHABET = string.ascii_lowercase
print(f"[*] Alphabet:\n{'.'*len(ALPHABET)}\n{ALPHABET}\n{'*'*len(ALPHABET)}")

[*] Alphabet:
..........................
abcdefghijklmnopqrstuvwxyz
**************************


### Utils

In [62]:
def _assert_alphabet(text):
    assert all(c in ALPHABET for c in text)

def alphabet_diff(a, b):
    """Compute (a - b) inside alphabet."""
    return ALPHABET[(ALPHABET.index(a) - ALPHABET.index(b)) % len(ALPHABET)]

def alphabet_sum(a, b):
    """Compute (a + b) inside alphabet."""
    return ALPHABET[(ALPHABET.index(a) + ALPHABET.index(b)) % len(ALPHABET)]

def text2message(text):
    return "".join(c for c in text if c in ALPHABET)

def print_enumerated(iterable, s):
    print(f'[*] {s}:')
    for i, item in enumerate(iterable, start=1):
        t = str(item)
        if len(t) > 100:
            print(f'{i: >2}: {t[:80]}...')
        else:
            print(f'{i: >2}: {t}')

### Vigener Algorithm

In [65]:
def _vigener_encrypt(message, key):
    """
    Encrypt message using `Vigener algorithm`_.
    
    Args:
        message (str): Message to encrypt.
        key (str): Secret key.
    
    Returns:
        str: Encrypted message.
    
    .. _Vigener algorithm:
        https://en.wikipedia.org/wiki/Vigenère_cipher
    """
    
    _assert_alphabet(message)
    _assert_alphabet(key)
    
    key_len = len(key)
    ciphertext = ''
    
    for i, mi in enumerate(message):
        # Note: mi + ki = ci
        # Note: ki = K[ki mod len(K)]
        ki = key[i % key_len]
        ci = alphabet_sum(mi, ki)
        ciphertext += ci
    
    return ciphertext

def vigener_encrypt(text, key, *, preserve_whitespace=False):
    if preserve_whitespace:
        wss = [m.start() for m in re.finditer(' ', text)]
        ciphertext = vigener_encrypt(text.replace(' ', ''), key)
        l = list(ciphertext)
        for p in wss:
            l.insert(p, ' ')
        return ''.join(l)
    else:
        return _vigener_encrypt(text2message(text), key)

### Test Vigener encryption

In [68]:
# Example from https://en.wikipedia.org/wiki/Vigenère_cipher
text = "ATTACK AT DAWN"
key = "LEMON"
answer = "LXFOPV EF RNHR"
ciphertext = vigener_encrypt(text.lower(), key.lower(), preserve_whitespace=True)

print(f"[*] Text: {text}")
print(f"[*] Key: {key}")
print(f"[+] Ciphertext: {ciphertext}")

assert ciphertext == answer, "Encryption failed"

[*] Text: ATTACK AT DAWN
[*] Key: LEMON
[+] Ciphertext: lxfopv ef rnhr


AssertionError: Encryption failed

### Encrypt some text

In [45]:
# Texts were generated by http://metaphorpsum.com/
# input_filename = "input-1-20.txt"
# input_filename = "input-1-30.txt"
input_filename = "New Text Document.txt"
# input_filename = "input-10-50.txt"
# input_filename = "input-4-50.txt"

In [69]:
with open(input_filename) as f:
    text = f.read()

message = text2message(text)
key = "adrar"
ciphertext = vigener_encrypt(text, key)

# with open("output.txt", "w") as f:
#     f.write(ciphertext)

print(f"[*] Text: {text[:70]}...")
print(f"[*] Message:    {message[:70]}...")
print(f"[+] Ciphertext: {ciphertext[:70]}...")

[*] Text: I am a third year student of computer science. I study information sec...
[*] Message:    amathirdyearstudentofcomputersciencestudyinformationsecurityandamhappy...
[+] Ciphertext: aprtyiuuyvaujtldhetffffmguwvrjclvntevkuuyleffrprtzoqjetuuztpaquadhdgpp...


### Kasiski Examination

In [70]:
def kasiski_examination(ciphertext, seq_len, max_key_len, *, verbose=False):
    """
    Find the most probable key length using `Kasiski examination`_.
    
    Args:
        ciphertext (str): Vigener-ciphered text.
        seq_len (int): Length of analyzed substrings.
        max_key_len (int): Upper bound for the key length.
        verbose (bool): Be verbose.
    
    Returns:
        int: The most probable key length.
    
    .. _Kasiski examination:
        https://en.wikipedia.org/wiki/Kasiski_examination
    """
    
    _assert_alphabet(ciphertext)

    # Find positions of each substring of length `seq_len`
    seq_positions = defaultdict(list)  # {seq: [pos]}
    for i in range(len(ciphertext) - seq_len):
        seq_positions[ciphertext[i : i + seq_len]].append(i)
    
    # Drop non-repeated sequences
    seq_positions = {
        seq: positions
        for seq, positions in seq_positions.items()
        if len(positions) >= 2
    }
    assert len(seq_positions) > 0, f"No repeated sequences of length {seq_len}"
    
    if verbose:
        print("[*] Positions:")
        for seq, positions in seq_positions.items():
            print(f"    {seq}: {positions}")

    # Calculte spacings between subsequent positions for each sequence
    seq_spacings = defaultdict(list)  # {seq: [space]}
    for seq, positions in seq_positions.items():
        for a, b in zip(positions, positions[1:]):
            seq_spacings[seq].append(b - a)
    
    if verbose:
        print("[*] Spacings:")
        for seq, spacings in seq_spacings.items():
            print(f"    {seq}: {spacings}")
    
    # Count factors (<=max_key_len) of all spacings
    factor_count = Counter()
    for spacings in seq_spacings.values():
        for space in spacings:
            for f in range(2, min(max_key_len, int(math.sqrt(space)) + 1)):
                if space % f == 0:
                    factor_count[f] += 1
    
    if verbose:
        print("[*] Factor counts:")
        for f, count in factor_count.items():
            print(f"    {f}: {count}")
    
    # Find the most probable key length
    key_len = factor_count.most_common()[0][0]
    
    if verbose:
        print(f"[+] The most probable key length: {key_len}")
    
    return key_len

### Perform Kasiski Examination

In [71]:
probable_key_len = kasiski_examination(ciphertext, 5, 10, verbose=True)

AssertionError: No repeated sequences of length 5

In [74]:
probable_key_len = kasiski_examination(ciphertext, 3, 10)
probable_key_len

3

### Frequence Analysis

In [75]:
def _freq(text):
    return Counter(text)

def _blocks(ciphertext, key_len):
    return [''.join(ciphertext[shift + i]
                    for i in range(0, len(ciphertext) - shift, key_len))
            for shift in range(key_len)]

def frequency_analysis(ciphertext, key_len, text_freq, *, verbose=False):
    """
    Find the most probable key using `frequency analysis`_.
    
    Args:
        ciphertext (str): Vigener-ciphered text.
        key_len (int): Key length.
        text_freq (collections.Counter): Estimated table of letter frequencies.
    
    Returns:
        str: The most probable key found by `frequency analysis`_.
    
    .. _frequency analysis:
        https://en.wikipedia.org/wiki/Frequency_analysis
    """
    
    _assert_alphabet(ciphertext)
    
    blocks = _blocks(ciphertext, key_len)
    block_freqs = [_freq(block) for block in blocks]
    most_common_letter = text_freq.most_common()[0][0]
    
    if verbose:
        print_enumerated(blocks, 'Blocks')
        print("[*] Block frequencies (most common):")
        for i, block_freq in enumerate(block_freqs, start=1):
            print(f"{i: >2}: {block_freq.most_common(7)}")
        print(f"[*] Most common letter in original text: '{most_common_letter}'")
    
    # Find the most probable key
    key = ''
    for i, block in enumerate(blocks):
        # Note: mi + ki = ci, then ki = ci - mi
        ci = block_freqs[i].most_common()[0][0]
        ki = alphabet_diff(ci, most_common_letter)
        key += ki
    
    if verbose:
        print(f"[+] Most probable key: '{key}'")
    
    return key

In [76]:
text_freq = _freq(message)
text_freq.most_common(7)

[('e', 13), ('t', 12), ('a', 7), ('m', 7), ('i', 7), ('r', 7), ('n', 7)]

In [79]:
probable_key = frequency_analysis(ciphertext, 5, text_freq, verbose=True)

[*] Blocks:
 1: aiadfuceyrouahtltgett
 2: puuhfwlvlpquqdrhwhfrh
 3: rujefvvkerjzugzdyeijd
 4: tyttmrnuftetapmeeeyy
 5: yvlfgjtufztpdpgezigj
[*] Block frequencies (most common):
 1: [('t', 4), ('a', 3), ('u', 2), ('e', 2), ('i', 1), ('d', 1), ('f', 1)]
 2: [('h', 4), ('u', 3), ('p', 2), ('f', 2), ('w', 2), ('l', 2), ('q', 2)]
 3: [('j', 3), ('e', 3), ('r', 2), ('u', 2), ('v', 2), ('z', 2), ('d', 2)]
 4: [('t', 5), ('e', 4), ('y', 3), ('m', 2), ('r', 1), ('n', 1), ('u', 1)]
 5: [('g', 3), ('f', 2), ('j', 2), ('t', 2), ('z', 2), ('p', 2), ('y', 1)]
[*] Most common letter in original text: 'e'
[+] Most probable key: 'pdfpc'


In [80]:
probable_key

'pdfpc'

### Decrypt message

In [15]:
def vigener_decrypt(ciphertext, key):
    """
    Decrypt Vigener-ciphered text.
    
    Args:
        ciphertext (str): Vigener-ciphered text.
        key (int): Secret key.
    
    Returns:
        str: Decrypted message.
    """
    
    key_len = len(key)
    message = ''
    
    for i, ci in enumerate(ciphertext):
        # Note: mi + ki = ci, then mi = ci - ki
        ki = key[i % key_len]
        mi = alphabet_diff(ci, ki)
        message += mi
    
    return message

In [16]:
decrypted_message = vigener_decrypt(ciphertext, probable_key)

print(f"[*] Text:       {text[:70]}...")
print(f"[*] Message:    {message[:70]}...")
print(f"[*] Ciphertext: {ciphertext[:70]}...")
print(f"[+] Decrypted:  {decrypted_message[:70]}...")

assert decrypted_message == message, "Decryption failed"

[*] Text:       Some posit the croaky wedge to be less than funded. One cannot separat...
[*] Message:    SOMEPOSITTHECROAKYWEDGETOBELESSTHANFUNDEDONECANNOTSEPARATEALGERIASFROM...
[*] Ciphertext: ECGWTAGCLXTSWJSMYSOIPUYLSNSFWWEHBSRRIHVIPCHWGMBHGXESJSVMHYSPSSLAEETLGQ...
[+] Decrypted:  SOMEPOSITTHECROAKYWEDGETOBELESSTHANFUNDEDONECANNOTSEPARATEALGERIASFROM...
